# Lab 4
## Arc I
### Mohsen Ahmadkhani

In [1]:
# Download temperature data from NDAWN
import requests
from urllib.parse import urlencode
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
import time

def get_weather(start_date, end_date, station_count = None):
    df = []
    row = 1
    for station in range(1, station_count):
        base_url = 'https://ndawn.ndsu.nodak.edu/table.csv?'
        params = f"station={str(station)}&begin_date={start_date}&end_date={end_date}&ttype=daily&quick_pick=&variable=ddmxt&variable=ddmnt&variable=ddavt&variable=ddbst&variable=ddtst&variable=ddws&variable=ddmxws&variable=ddsr&variable=ddr&variable=dddp&variable=ddwc"
        url = base_url+params
        r = requests.get(url)

        with open("response.csv", "w") as text_file:
            text_file.write(r.content.decode('utf-8'))

        if row == 1:
            instance = pd.read_csv('response.csv', header=[0,1],skiprows=3)
            if instance.iloc[0][0].find('<') == -1: 
                df = instance
                row = 0
        else:
            instance = pd.read_csv('response.csv', header=[0,1],skiprows=3)
            df = pd.concat([df, instance], ignore_index=True)
    
    return df
            


today_date = list(time.localtime())
yesterday = str(today_date[0]) + '-' + str(today_date[1]) + '-' + str(today_date[2]-1)
last_month = str(today_date[0]) + '-' + str(today_date[1]-1) + '-' + str(today_date[2]-1)

start_date = last_month
end_date = yesterday

df = get_weather(start_date, end_date, 300) # get the data for all ND weather stations
df.columns = [f'{i}_{j}' for i, j in df.columns] # concat two header rows into one 

# df.to_csv('weather_monthly.csv', encoding='utf-8', index=False)

stations_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude_deg, df.Latitude_deg))
stations_gdf.crs = 4326

stations_gdf.head()




ModuleNotFoundError: No module named 'geopandas'

In [7]:
!pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)
  Using cached Fiona-1.8.20.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.19.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.18.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.17.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.16.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.15.tar.gz (1.3 MB)

  ERROR: Command errored out with exit status 1:
   command: 'C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\UMN-AH~1\\AppData\\Local\\Temp\\pip-install-pam_k3he\\fiona_b4c62dc81dc54a44a8cccc07e3ca16e7\\setup.py'"'"'; __file__='"'"'C:\\Users\\UMN-AH~1\\AppData\\Local\\Temp\\pip-install-pam_k3he\\fiona_b4c62dc81dc54a44a8cccc07e3ca16e7\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\UMN-AH~1\AppData\Local\Temp\pip-pip-egg-info-dgoor45_'
       cwd: C:\Users\UMN-AH~1\AppData\Local\Temp\pip-install-pam_k3he\fiona_b4c62dc81dc54a44a8cccc07e3ca16e7\
  Complete output (1 lines):
  A GDAL API version must be specified. Provide a path to gdal-conf


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.14.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.13.post1.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.13.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.12.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.11.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using cached Fiona-1.8.10.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Using 

In [ ]:
#aggregate by averaging the average daily temperature to get to the monthly average
stations_gdf = stations_gdf.groupby('Station Name_Unnamed: 0_level_1').mean() 
stations_gdf = gpd.GeoDataFrame(stations_gdf, geometry=gpd.points_from_xy(stations_gdf.Longitude_deg, stations_gdf.Latitude_deg))
stations_gdf.crs = 4326
stations_gdf = stations_gdf.reset_index()
stations_gdf


In [ ]:
# plot an interactive map
import warnings
warnings.filterwarnings('ignore') 

m = folium.Map(location = [44.9779006, -93.1831530], tiles='OpenStreetMap' , zoom_start = 12) # tiles="Stamen Toner"

for _, r in stations_gdf.iterrows():

    sim_geo = gpd.GeoSeries(r['geometry']) 
    geo_j = sim_geo.to_json()
   
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function = lambda x: {'color': 'red', 'weight': 1,  'fillColor': 'YlGnBu'})
    folium.Popup(f"<i>Station: {r['Station Name_Unnamed: 0_level_1']}, Average Monthly Temperature: {r['Avg Temp_Degrees F']} °F</i>").add_to(geo_j)
    folium.Tooltip(f"<i>Station: {r['Station Name_Unnamed: 0_level_1']} <br> Avg Monthly Temperature: {r['Avg Temp_Degrees F']} °F</i>").add_to(geo_j)
    geo_j.add_to(m)


sw = [44.867066, -104.248]
ne = [48.884, -93.065245]

m.fit_bounds([sw, ne]) 

m



In [ ]:
df = geopandas.GeoDataFrame(stations_gdf, geometry='geom')
df.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
df.to_file('ndawn_weather.shp', driver='ESRI Shapefile')

In [3]:
import arcpy
# dem_3m = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb\dem_3m_m'
# p = r'C:\Users\umn-ahmad178\Desktop\arc1lab2\elevation_data.gdb'
p = r'C:\Users\umn-ahmad178\Desktop\lab4arc1'
arcpy.env.workspace = p  # setting workspace to the current directory
arcpy.env.overwriteOutput = True


In [6]:
arcpy.ListFeatureClasses()

['ndawn_weather.shp']

In [ ]:


Idw(in_point_features, z_field, {cell_size}, {power}, {search_radius}, {in_barrier_polyline_features})



In [9]:
#IDW
from arcpy.sa import *

# Set local variables
inPointFeatures = 'ndawn_weather.shp'
zField = "Avg Temp F"
cellSize = 0.014477764
power = 2
searchRadius = RadiusVariable(10, 150000)

# Execute IDW
outIDW = Idw(inPointFeatures, zField, cellSize, power, searchRadius)

# Save the output 

outIDW.save(r'C:\Users\umn-ahmad178\Desktop\lab4arc1\idwout.tif')
outKrig.save(r"C:\Users\umn-ahmad178\Desktop\lab4arc1\krigout.tif")

In [12]:
# Universal Kriging
outKrig = Kriging(inPointFeatures, zField, KrigingModelOrdinary("Linear drift", 0.014477764, 2.6, 542, 0), 2000, RadiusFixed(20000, 1))
outKrig.save(r"C:\Users\umn-ahmad178\Desktop\lab4arc1\uni_krigout.tif")


In [13]:
# Ordinary Kriging
outKrig = Kriging(inPointFeatures, zField, KrigingModelOrdinary("GAUSSIAN", 0.014477764, 2.6, 542, 0), 2000, RadiusFixed(20000, 1))
outKrig.save(r"C:\Users\umn-ahmad178\Desktop\lab4arc1\krigout_ordinary.tif")


In [ ]:
arcpy.LocalPolynomialInterpolation_ga(
    inPointFeatures, zField, "outLPI", r"C:\Users\umn-ahmad178\Desktop\lab4arc1\idwout.tif\polynomial.tif", "2000",
    "2", arcpy.SearchNeighborhoodSmooth(300000, 300000, 0, 0.5), "QUARTIC", 
    "", "", "", "", "PREDICTION")

